In [72]:
!pip install -q opencv-python
!pip install -q git+https://github.com/tensorflow/docs
!pip install tensorflow-hub
!pip install opencv-python
!pip install tensorflow
!pip install realesrgan
!pip install yt-dlp

In [73]:
import tensorflow_hub as hub
import cv2
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import cv2
import yt_dlp
from IPython.display import HTML, display, Image
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
import imageio

In [74]:
def download_video_yt_dlp(url, output_path):
    # Set download options to download only a single format (MP4 or WebM)
    ydl_opts = {
        'outtmpl': output_path,  # Define the output file name
        'format': 'best',  # Downloads the best available format that includes both video and audio
    }
    
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])
    print(f"Video downloaded to: {output_path}")

# Function to extract frames from video (if you want to process them later)
def extract_frames(video_path, num_frames=100):
    cap = cv2.VideoCapture(video_path)
    frames = []
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_interval = total_frames // num_frames
    for i in range(num_frames):
        ret, frame = cap.read()
        if not ret:
            break
        if i % frame_interval == 0:
            frames.append(frame)
    cap.release()
    return np.array(frames)

# Function to display the video in a Jupyter notebook
def display_video(video_path):
    video_tag = f"""
    <video width="640" height="480" controls>
      <source src="{video_path}" type="video/mp4">
      Your browser does not support the video tag.
    </video>
    """
    display(HTML(video_tag))

# Example usage
url = "https://www.youtube.com/watch?v=i0UsldY4Ue4"  # Replace with your video URL
download_video_yt_dlp(url, 'downloaded_video.mp4')

# Display the video in the Jupyter notebook
display_video('downloaded_video.mp4')

[youtube] Extracting URL: https://www.youtube.com/watch?v=i0UsldY4Ue4
[youtube] i0UsldY4Ue4: Downloading webpage
[youtube] i0UsldY4Ue4: Downloading ios player API JSON
[youtube] i0UsldY4Ue4: Downloading mweb player API JSON
[youtube] i0UsldY4Ue4: Downloading m3u8 information
[info] i0UsldY4Ue4: Downloading 1 format(s): 18
[download] downloaded_video.mp4 has already been downloaded
[download] 100% of    5.27MiB
Video downloaded to: downloaded_video.mp4


In [75]:
interpreter = tf.lite.Interpreter(model_path='thunder3.tflite')
interpreter.allocate_tensors()

In [76]:
import collections

confidence_threshold = 0.3

EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 2)


# Dictionary to store total movement for each keypoint
movement_data = collections.defaultdict(float)

# Dictionary to store confidence scores for each keypoint
confidence_data = collections.defaultdict(list)

def draw_keypoints_with_data(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y, x, 1]))
    
    for i, kp in enumerate(shaped):
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            # Draw the keypoint
            cv2.circle(frame, (int(kx), int(ky)), 4, (0, 255, 0), -1)
            # Display coordinates and score
            cv2.putText(frame, f"{i}: {kp_conf:.2f}",
                        (int(kx), int(ky) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1)

        
def calculate_movement(previous_keypoints, current_keypoints, confidence_threshold):
    if previous_keypoints is None:
        return None  # First frame, no movement to calculate
    
    for i, (prev, curr) in enumerate(zip(previous_keypoints, current_keypoints)):
        py, px, pc = prev
        cy, cx, cc = curr
        
        if pc > confidence_threshold and cc > confidence_threshold:
            distance = np.sqrt((cx - px)**2 + (cy - py)**2)
            movement_data[i] += distance
            confidence_data[i].append(cc)

In [77]:
def rotate_video(input_path, output_path, rotation=cv2.ROTATE_90_CLOCKWISE):
    """
    Rotate a video by a specified angle and save the output.
    
    :param input_path: Path to the input video.
    :param output_path: Path to save the rotated video.
    :param rotation: OpenCV rotation flag (default: cv2.ROTATE_90_CLOCKWISE).
    """
    cap = cv2.VideoCapture(input_path)
    if not cap.isOpened():
        print(f"Error: Unable to open video file {input_path}")
        return
    
    # Get video properties
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))  # Swapped height and width
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for MP4 format
    
    # Create VideoWriter to save the rotated video
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Rotate the frame
        rotated_frame = cv2.rotate(frame, rotation)
        
        # Write the rotated frame to the output video
        out.write(rotated_frame)
    
    cap.release()
    out.release()
    print(f"Rotated video saved to: {output_path}")

# Rotate the video
input_video = 'downloaded_video.mp4'
rotated_video = 'rotated_video.mp4'
rotate_video(input_video, rotated_video)

# Display the rotated video 
display_video(rotated_video)
cap = cv2.VideoCapture('rotated_video.mp4')
previous_keypoints = None

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break  # Exit the loop if the video has ended
    
    # Reshape image
    img = frame.copy()
    img = tf.image.resize_with_pad(np.expand_dims(img, axis=0), 256, 256)
    input_image = tf.cast(img, dtype=tf.float32)
    
    # Setup input and output
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    
    # Make predictions
    interpreter.set_tensor(input_details[0]['index'], np.array(input_image))
    interpreter.invoke()
    keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])[0][0]

    draw_connections(frame, keypoints_with_scores, EDGES, confidence_threshold)
    draw_keypoints_with_data(frame, keypoints_with_scores, confidence_threshold)
    
    cv2.imshow('MoveNet Thunder with Data', frame)
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Rotated video saved to: rotated_video.mp4


In [78]:
import os
import pandas as pd



df = pd.read_csv('URL_pose_dataset.csv')


def download_video_yt_dlp(url, output_path):
    if os.path.exists(output_path):
        print(f"Video already exists: {output_path}")
    else:
        ydl_opts = {
            'outtmpl': output_path,
            'format': 'best',
            'continue': True,  
        }
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([url])

def download_videos_from_csv(csv_file, output_dir):
    failed_downloads = []
    for idx, row in df.iterrows():
        url = row['url']
        output_path = f"{output_dir}/video_{idx+1}.mp4"
        
        try:
            download_video_yt_dlp(url, output_path)
        except Exception as e:
            failed_downloads.append(idx+1)  
            
    return failed_downloads


failed_downloads = download_videos_from_csv('URL_pose_dataset.csv', 'videos')


if failed_downloads:
    print(f"Videos failed to download from rows: {failed_downloads}")
else:
    print("All videos were downloaded successfully.")



Video already exists: videos/video_1.mp4
Video already exists: videos/video_2.mp4
Video already exists: videos/video_3.mp4
Video already exists: videos/video_4.mp4
Video already exists: videos/video_5.mp4
Video already exists: videos/video_6.mp4
[youtube] Extracting URL: https://www.youtube.com/watch?v=RU5DWogrC8k
[youtube] RU5DWogrC8k: Downloading webpage
[youtube] RU5DWogrC8k: Downloading ios player API JSON
[youtube] RU5DWogrC8k: Downloading mweb player API JSON


ERROR: [youtube] RU5DWogrC8k: This video has been removed for violating YouTube's Terms of Service


[youtube] Extracting URL: https://www.youtube.com/watch?v=eiYANQY5NxM
[youtube] eiYANQY5NxM: Downloading webpage
[youtube] eiYANQY5NxM: Downloading ios player API JSON
[youtube] eiYANQY5NxM: Downloading mweb player API JSON


ERROR: [youtube] eiYANQY5NxM: Private video. Sign in if you've been granted access to this video. Use --cookies-from-browser or --cookies for the authentication. See  https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp  for how to manually pass cookies. Also see  https://github.com/yt-dlp/yt-dlp/wiki/Extractors#exporting-youtube-cookies  for tips on effectively exporting YouTube cookies


Video already exists: videos/video_9.mp4
Video already exists: videos/video_10.mp4
Video already exists: videos/video_11.mp4
[youtube] Extracting URL: https://www.youtube.com/watch?v=nZtPMXCtJDc
[youtube] nZtPMXCtJDc: Downloading webpage
[youtube] nZtPMXCtJDc: Downloading ios player API JSON
[youtube] nZtPMXCtJDc: Downloading mweb player API JSON


...

[youtube] Extracting URL: https://www.youtube.com/watch?v=hcpptfftDcA
[youtube] hcpptfftDcA: Downloading webpage
[youtube] hcpptfftDcA: Downloading ios player API JSON
[youtube] hcpptfftDcA: Downloading mweb player API JSON


ERROR: [youtube] hcpptfftDcA: Private video. Sign in if you've been granted access to this video. Use --cookies-from-browser or --cookies for the authentication. See  https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp  for how to manually pass cookies. Also see  https://github.com/yt-dlp/yt-dlp/wiki/Extractors#exporting-youtube-cookies  for tips on effectively exporting YouTube cookies


Video already exists: videos/video_87.mp4
Video already exists: videos/video_88.mp4
Video already exists: videos/video_89.mp4
Video already exists: videos/video_90.mp4
Video already exists: videos/video_91.mp4
[youtube] Extracting URL: https://www.youtube.com/watch?v=KaRV_QbrO6M
[youtube] KaRV_QbrO6M: Downloading webpage
[youtube] KaRV_QbrO6M: Downloading ios player API JSON
[youtube] KaRV_QbrO6M: Downloading mweb player API JSON


ERROR: [youtube] KaRV_QbrO6M: This video has been removed for violating YouTube's Community Guidelines


Video already exists: videos/video_93.mp4
Video already exists: videos/video_94.mp4
Video already exists: videos/video_95.mp4
Videos failed to download from rows: [7, 8, 12, 18, 22, 23, 25, 26, 27, 28, 29, 30, 39, 42, 45, 47, 53, 55, 60, 61, 66, 67, 68, 69, 70, 71, 74, 75, 85, 86, 92]


In [79]:
!pip install moviepy==1.0.3
!pip install --upgrade moviepy
!pip install --upgrade imageio decorator

  Using cached moviepy-1.0.3-py3-none-any.whl
  Using cached decorator-4.4.2-py2.py3-none-any.whl.metadata (4.2 kB)
Using cached decorator-4.4.2-py2.py3-none-any.whl (9.2 kB)
  Attempting uninstall: decorator
    Found existing installation: decorator 5.1.1
    Uninstalling decorator-5.1.1:
      Successfully uninstalled decorator-5.1.1
  Attempting uninstall: moviepy
    Found existing installation: moviepy 2.1.2
    Uninstalling moviepy-2.1.2:
      Successfully uninstalled moviepy-2.1.2


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.4.3 requires jedi<0.19.0,>=0.17.2, but you have jedi 0.19.1 which is incompatible.


  Using cached moviepy-2.1.2-py3-none-any.whl.metadata (6.9 kB)
Using cached moviepy-2.1.2-py3-none-any.whl (126 kB)
  Attempting uninstall: moviepy
    Found existing installation: moviepy 1.0.3
    Uninstalling moviepy-1.0.3:
      Successfully uninstalled moviepy-1.0.3


  Using cached decorator-5.1.1-py3-none-any.whl.metadata (4.0 kB)
Using cached decorator-5.1.1-py3-none-any.whl (9.1 kB)
  Attempting uninstall: decorator
    Found existing installation: decorator 4.4.2
    Uninstalling decorator-4.4.2:
      Successfully uninstalled decorator-4.4.2


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.4.3 requires jedi<0.19.0,>=0.17.2, but you have jedi 0.19.1 which is incompatible.


In [80]:
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip

df = pd.read_csv('URL_pose_dataset.csv')

def trim_video(input_path, output_path, start_min, start_sec, end_min, end_sec):
    try:
        start_time = start_min * 60 + start_sec
        end_time = end_min * 60 + end_sec
        
        print(f"Processing video: {input_path} (Start: {start_time}s, End: {end_time}s)")
        
        
        ffmpeg_extract_subclip(input_path, start_time, end_time, output_path)  
        
        print(f"Trimmed video saved to: {output_path}")
    except Exception as e:
        print(f"Error trimming video {input_path}: {e}")

def process_videos(csv_file, input_dir, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    for idx, row in df.iterrows():
        input_path = os.path.join(input_dir, f"video_{idx+1}.mp4")
        output_path = os.path.join(output_dir, f"video_{idx+1}_trimmed.mp4")
        
        
        if os.path.exists(input_path):
            trim_video(
                input_path, 
                output_path, 
                row['start_min'], 
                row['start_sec'], 
                row['end_min'], 
                row['end_sec']
            )
        else:
            print(f"Video file not found: {input_path}")

process_videos('URL_pose_dataset.csv', 'videos', 'trimmed_videos')



Processing video: videos\video_1.mp4 (Start: 0.0s, End: 44.0s)
MoviePy - Running:
>>> C:\Users\user\anaconda3\Lib\site-packages\imageio_ffmpeg\binaries\ffmpeg-win-x86_64-v7.1.exe -y -ss 0.00 -i videos\video_1.mp4 -t 44.00 -map 0 -vcodec copy -acodec copy -copyts trimmed_videos\video_1_trimmed.mp4
MoviePy - Command successful
Trimmed video saved to: trimmed_videos\video_1_trimmed.mp4
Processing video: videos\video_2.mp4 (Start: 0.0s, End: 153.0s)
MoviePy - Running:
>>> C:\Users\user\anaconda3\Lib\site-packages\imageio_ffmpeg\binaries\ffmpeg-win-x86_64-v7.1.exe -y -ss 0.00 -i videos\video_2.mp4 -t 153.00 -map 0 -vcodec copy -acodec copy -copyts trimmed_videos\video_2_trimmed.mp4
MoviePy - Command successful
Trimmed video saved to: trimmed_videos\video_2_trimmed.mp4
Processing video: videos\video_3.mp4 (Start: 0.0s, End: 178.0s)
MoviePy - Running:
>>> C:\Users\user\anaconda3\Lib\site-packages\imageio_ffmpeg\binaries\ffmpeg-win-x86_64-v7.1.exe -y -ss 0.00 -i videos\video_3.mp4 -t 178.00 -m

In [81]:
import shutil


input_dir = 'trimmed_videos'
output_dir = 'rotated_and_cut_videos'


if not os.path.exists(output_dir):
    os.makedirs(output_dir)

for video_filename in os.listdir(input_dir):
    if video_filename.endswith('.mp4'):  
        input_video = os.path.join(input_dir, video_filename)
        output_video = os.path.join(output_dir, f"rotated_{video_filename}")
        
        
        rotate_video(input_video, output_video)

print("All videos processed.")


Rotated video saved to: rotated_and_cut_videos\rotated_video_10_trimmed.mp4
Error: Unable to open video file trimmed_videos\video_11_trimmed.mp4
Error: Unable to open video file trimmed_videos\video_13_trimmed.mp4
Error: Unable to open video file trimmed_videos\video_14_trimmed.mp4
Error: Unable to open video file trimmed_videos\video_15_trimmed.mp4
Error: Unable to open video file trimmed_videos\video_16_trimmed.mp4
Error: Unable to open video file trimmed_videos\video_17_trimmed.mp4
Rotated video saved to: rotated_and_cut_videos\rotated_video_19_trimmed.mp4
Rotated video saved to: rotated_and_cut_videos\rotated_video_1_trimmed.mp4
Rotated video saved to: rotated_and_cut_videos\rotated_video_20_trimmed.mp4
Error: Unable to open video file trimmed_videos\video_21_trimmed.mp4
Rotated video saved to: rotated_and_cut_videos\rotated_video_24_trimmed.mp4
Rotated video saved to: rotated_and_cut_videos\rotated_video_2_trimmed.mp4
Error: Unable to open video file trimmed_videos\video_31_trimme